# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
import pandas as pd

# your code here
data = pd.read_csv(r"training.1600000.processed.noemoticon.csv",encoding = "ISO-8859-1")
df = data.sample(n=20000,random_state =123,axis =0)
df.head()
col_names = ['target', 'id', 'date', 'flag', 'user', 'text']


In [2]:
df.columns = col_names
df

,target,id,date,flag,user,text
448282,0,2068921344,Sun Jun 07 14:56:43 PDT 2009,NO_QUERY,gemmagreeb,so tired ive had like 7hrs sleep the past 2 n...
648495,0,2237041125,Fri Jun 19 05:04:02 PDT 2009,NO_QUERY,gopaldass,shucks... missed out on a key event this week ...
132529,0,1835775067,Mon May 18 06:43:29 PDT 2009,NO_QUERY,suedahlgren,@Geekdom Me too! Where are those nice 75 degre...
182348,0,1967122225,Fri May 29 19:00:48 PDT 2009,NO_QUERY,sarahleefitz,Its Spikey Dikey's last Friday night in The Bu...
907612,4,1695846131,Mon May 04 07:04:29 PDT 2009,NO_QUERY,Digidreams,good mornin ? http://blip.fm/~5jryt
...,...,...,...,...,...,...
1503733,4,2072031742,Sun Jun 07 20:17:03 PDT 2009,NO_QUERY,Gary200,@Richochet: Thanks! You are awesome too.
912756,4,1752385539,Sat May 09 21:46:48 PDT 2009,NO_QUERY,cookie_ninja,About to leave the fights. I'm tired and my vo...
1398819,4,2054152487,Sat Jun 06 06:41:19 PDT 2009,NO_QUERY,MyMeatWorks,@FuurHunt Thanks for the support!
422592,0,2062681668,Sat Jun 06 23:58:09 PDT 2009,NO_QUERY,idoodleallday,@LisaBBTCullen OMG I wish I were in NYC to see...


In [3]:
df["target"].value_counts()

0    10010
4     9990
Name: target, dtype: int64

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [4]:
df.reset_index(drop = True,inplace =True)

In [5]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
import re
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Denish\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
# your code here
def clean_up(s):
    pattern_urls = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    cleaned_string = re.sub(pattern_urls, '',s)
    cleaned_string = cleaned_string.lower()
    cleaned_string = re.sub(r'[^a-zA-Z\s]'," ", cleaned_string)
    return cleaned_string  

In [7]:
df["text_processed"] = df["text"].apply(clean_up)

In [8]:
def tokenize(s):
    tokens = word_tokenize(s)
    tokens = [word for word in tokens if word.isalnum()]
    return tokens

In [9]:
df["text_processed"] = df["text_processed"].apply(tokenize)

In [10]:
def stem_and_lemmatize(l):
    ps = PorterStemmer()
    stemmed = [ps.stem(w) for w in l]
    lemmatizer = WordNetLemmatizer() 
    lemmatized = [lemmatizer.lemmatize(word) for word in stemmed]
    return lemmatized

In [11]:
df["text_processed"] = df["text_processed"].apply(stem_and_lemmatize)

In [12]:
def remove_stopwords(l):
    lemmatized_wt_sw = [word for word in l if not word in stopwords.words('english')]
    return lemmatized_wt_sw

In [13]:
df["text_processed"] = df["text_processed"].apply(remove_stopwords)

In [14]:
df.head()

,target,id,date,flag,user,text,text_processed
0,0,2068921344,Sun Jun 07 14:56:43 PDT 2009,NO_QUERY,gemmagreeb,so tired ive had like 7hrs sleep the past 2 n...,"[tire, ive, like, hr, sleep, past, night, gooo..."
1,0,2237041125,Fri Jun 19 05:04:02 PDT 2009,NO_QUERY,gopaldass,shucks... missed out on a key event this week ...,"[shuck, miss, key, event, thi, week, gt, gt, m..."
2,0,1835775067,Mon May 18 06:43:29 PDT 2009,NO_QUERY,suedahlgren,@Geekdom Me too! Where are those nice 75 degre...,"[geekdom, nice, degre, day, could, use, week, ..."
3,0,1967122225,Fri May 29 19:00:48 PDT 2009,NO_QUERY,sarahleefitz,Its Spikey Dikey's last Friday night in The Bu...,"[spikey, dikey, last, friday, night, burg, way..."
4,4,1695846131,Mon May 04 07:04:29 PDT 2009,NO_QUERY,Digidreams,good mornin ? http://blip.fm/~5jryt,"[good, mornin, jryt]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [15]:
# your code here
from nltk import FreqDist
text_processed = FreqDist()

from nltk import FreqDist
for i in df["text_processed"]:
    
    text_processed.update(i)
   

In [16]:
all_words = text_processed.most_common(5000)

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [17]:
all_words = dict(all_words)

In [18]:
# your code here
word_Features = list(all_words.keys())


In [19]:
def find_features(document):
    words = set(document)
    features = {}
    for w in word_Features:
        features[w] = (w in words)
    return features    
        

In [20]:
df["text_processed"]= df["text_processed"].apply(find_features)

In [21]:
df["target"] = df["target"].replace({0:False, 4:True})

In [22]:
df["target"]

0        False
1        False
2        False
3        False
4         True
         ...  
19995     True
19996     True
19997     True
19998    False
19999     True
Name: target, Length: 20000, dtype: bool

In [ ]:
# is_positive = df["target"]
# df.drop("target",axis =1,inplace = True)

In [23]:
df.head()

,target,id,date,flag,user,text,text_processed
0,False,2068921344,Sun Jun 07 14:56:43 PDT 2009,NO_QUERY,gemmagreeb,so tired ive had like 7hrs sleep the past 2 n...,"{'go': False, 'get': False, 'wa': False, 'day'..."
1,False,2237041125,Fri Jun 19 05:04:02 PDT 2009,NO_QUERY,gopaldass,shucks... missed out on a key event this week ...,"{'go': False, 'get': False, 'wa': False, 'day'..."
2,False,1835775067,Mon May 18 06:43:29 PDT 2009,NO_QUERY,suedahlgren,@Geekdom Me too! Where are those nice 75 degre...,"{'go': False, 'get': False, 'wa': False, 'day'..."
3,False,1967122225,Fri May 29 19:00:48 PDT 2009,NO_QUERY,sarahleefitz,Its Spikey Dikey's last Friday night in The Bu...,"{'go': False, 'get': False, 'wa': False, 'day'..."
4,True,1695846131,Mon May 04 07:04:29 PDT 2009,NO_QUERY,Digidreams,good mornin ? http://blip.fm/~5jryt,"{'go': False, 'get': False, 'wa': False, 'day'..."


In [27]:
columns_list = df[['text_processed', 'target']].values.tolist()

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [29]:
# your code here
train_set, test_set = columns_list[10000:], columns_list[:10000]
classifier = nltk.NaiveBayesClassifier.train(train_set)

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [30]:
# your code here
print(nltk.classify.accuracy(classifier, test_set))

0.7241


In [33]:
classifier.show_most_informative_features(10)

Most Informative Features
                     sad = True            False : True   =     22.7 : 1.0
                 headach = True            False : True   =     22.1 : 1.0
                   smile = True             True : False  =     17.3 : 1.0
                    crap = True            False : True   =     16.7 : 1.0
                     www = True             True : False  =     10.8 : 1.0
                unfortun = True            False : True   =     10.5 : 1.0
                   magic = True             True : False  =     10.1 : 1.0
                   shout = True             True : False  =     10.1 : 1.0
                 depress = True            False : True   =     10.0 : 1.0
                    sore = True            False : True   =     10.0 : 1.0


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here